In [2]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
import keras

## 整理数据集

In [9]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
val_numb = -1000
X_train = X_train[:val_numb]
y_train = y_train[:val_numb]
X_train_val = X_train[val_numb:]
y_train_val = y_train[val_numb:]

X_train = X_train[:700]
y_train = y_train[:700]

# data pre-processing
X_train = X_train.reshape(-1, 1,28, 28)/255.
X_test = X_test.reshape(-1, 1,28, 28)/255.
X_train_val = X_train_val.reshape(-1, 1,28, 28)/255.
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)
y_train_val = np_utils.to_categorical(y_train_val, num_classes=10)

## 搭建网络

In [4]:
model = Sequential()
# 卷积层
model.add(Convolution2D(
    batch_input_shape=(None, 1, 28, 28),
    filters=32,
    kernel_size=5,
    strides=1,
    padding='same',     # Padding method
    data_format='channels_first',
    name="conv2D_1",
)) 
model.add(Activation('relu', name="activation_rule_1"))

# 池化层
model.add(MaxPooling2D(
    pool_size=2,
    strides=2,
    padding='same',
    data_format='channels_first',
    name="maxpooling2D_1"
))

model.add(Convolution2D(64, 5, strides=1, padding='same', data_format='channels_first', name="covn2D_2"))
model.add(Activation('relu', name="activation_rule_2"))
model.add(MaxPooling2D(2, 2, 'same', data_format='channels_first'))

# 经过以上处理之后数据shape为（64，7，7），需要将数据抹平成一维，再添加全连接层1
model.add(Flatten())
model.add(Dense(1024, name="dense_1"))
model.add(Activation('relu', name="activation_rule_3"))

# 输出层
model.add(Dense(10, name="dense_2"))
model.add(Activation('softmax', name="activation_softmax_4"))

# 优化方法，Loss函数 
adam = Adam(lr=1e-4)
model.compile(
    optimizer=adam,
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

W0808 09:30:19.942350 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 09:30:19.998867 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 09:30:20.018558 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 09:30:20.055052 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2D_1 (Conv2D)            (None, 32, 28, 28)        832       
_________________________________________________________________
activation_rule_1 (Activatio (None, 32, 28, 28)        0         
_________________________________________________________________
maxpooling2D_1 (MaxPooling2D (None, 32, 14, 14)        0         
_________________________________________________________________
covn2D_2 (Conv2D)            (None, 64, 14, 14)        51264     
_________________________________________________________________
activation_rule_2 (Activatio (None, 64, 14, 14)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 7, 7)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
__________

## Tensorboard可视化

In [10]:
# 引入Tensorboard
from keras.callbacks import TensorBoard
# tbCallBack = TensorBoard(log_dir='logs/101',  # log 目录
#                  histogram_freq=1000,  # 按照何等频率（epoch）来计算直方图，0为不计算
#                  batch_size=32,     # 用多大量的数据计算直方图
#                  write_graph=True,  # 是否存储网络结构图
#                  write_grads=True, # 是否可视化梯度直方图
#                  write_images=True,# 是否可视化参数
#                  embeddings_freq=0, 
#                  embeddings_layer_names=None, 
#                  embeddings_metadata=None)

tbCallBack = TensorBoard(log_dir='logs/101',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
                 batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True, # 是否可视化梯度直方图
                 write_images=True,# 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)

model.fit(X_train, y_train, epochs=1, batch_size=64, validation_data=(X_train_val, y_train_val), callbacks=[tbCallBack])

Train on 700 samples, validate on 1000 samples
Epoch 1/1
700/700 [==============================] - 4s 5ms/step - loss: 0.0989 - acc: 0.9800 - val_loss: 0.0361 - val_acc: 0.9860


In [8]:
loss, accuracy = model.evaluate(X_test, y_test)
print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

10000/10000 [==============================] - 13s 1ms/step

test loss:  0.08645022575296461

test accuracy:  0.9749
